**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [25]:
import ffmpeg

In [26]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

# Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [27]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. 

In [28]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model
        sauvegarder dans un fichier json """
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass
    

***
Why is ```epsilon``` essential?

Our agent will randomly select its action at first by a certain percentage, called ‘exploration rate’ that we call ‘epsilon’. It is essential for the agent to try all kinds of actions before it starts to see the patterns. When it is not deciding the action randomly, the agent will predict the reward value based on the current state and pick the action that will give the highest reward.
Here we have to implement a new function learning rate overriting this one to start the learning(exploitation) after exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following 

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        #using method act of agent
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        #keep the previous state
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        #mise à jour des connaissances et de l'apprentissage
        #weight du reseau de neuronnes?
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [29]:
b = np.zeros((10, 10,3))+128

In [30]:
board = np.zeros((10,10))
board[1][4]=1

In [31]:
b[board>0, 2]

array([128.])

In [32]:
b[board>0,0] = 256
b[board < 0, 2] = 256

In [33]:
board

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
#b[-2:,:,:]=0


In [35]:
b[:,-2:,:]=0
b[:2,:,:]=0
b[:,:2,:]=0
#b

In [36]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4 #increasing plateau of 2 cells
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.board = np.zeros((grid_size,grid_size))
        #position eau et position du rat
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        #for screening
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends.
        l'environnement recoit l'action et se modifie"""

        self.get_frame(int(self.t))
        
        #reinitialisation de la position du rat à chaque partie mais pas le bord
        self.position = np.zeros((self.grid_size, self.grid_size))
        
        #eau autour du board d'action
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        #differentes positions possibles
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        #input for the neural network
        #creation tuple ==> stock position and board
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        #take only the environment of the rat (everything the rat can see)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        #generation de la position du rat à au moins 3 cases de l'eau
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        #bonus for creation of board
        #distribution malus and bonus==> cheese and poison with binomial distribution with temperature 0.5
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        #cas où on aurait -0.5 , cheese preponderant
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [37]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=21 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

The use of the arrays ```position``` and ```board```.

position : this variable gives to the rat its position and the limit of the map (2 cells from the last limit) 
board : This is where we play. This variable indicates the position of the rewards (bonus/cheese and malus/poison are distributed on the board 14x14)

## Random Agent

***
The random Agent


In [38]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]
        

***
***
The game moves. You need to fill in the following function for the evaluation:

In [39]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)
            

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [40]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/12.0. Average score (-2.0)
Win/lose count 8.5/10.0. Average score (-1.75)
Win/lose count 11.5/15.0. Average score (-2.3333333333333335)
Win/lose count 8.0/16.0. Average score (-3.75)
Win/lose count 10.5/16.0. Average score (-4.1)
Win/lose count 14.5/11.0. Average score (-2.8333333333333335)
Win/lose count 12.5/16.0. Average score (-2.9285714285714284)
Win/lose count 5.5/14.0. Average score (-3.625)
Win/lose count 8.5/8.0. Average score (-3.1666666666666665)
Win/lose count 4.5/14.0. Average score (-3.8)
Win/lose count 11.0/11.0. Average score (-3.4545454545454546)
Win/lose count 10.5/11.0. Average score (-3.2083333333333335)
Win/lose count 11.0/9.0. Average score (-2.8076923076923075)
Win/lose count 7.0/21.0. Average score (-3.607142857142857)
Win/lose count 12.5/14.0. Average score (-3.466666666666667)
Win/lose count 9.0/19.0. Average score (-3.875)
Win/lose count 10.5/19.0. Average score (-4.147058823529412)
Win/lose count 11.5/12.0. Average score (-3.9444444444444

***
## DQN

Let us assume here that $T=\infty$.

***
Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
We implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. We specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [41]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        #if len(memory)>self.max_memory:
        #   self.memory.remove(self.memory[0])
        self.memory.append(m)
            
        

    def random_access(self):
        #print(np.random.sample(self.memory,16))
        #print(len(np.random.sample(len(self.memory))))
        idx = np.random.randint(0, len(self.memory))
        
        #print(minibatch)
        return self.memory[idx]
        #random.choice(self.memory)
        

***
The pipeline we will use for training is given below:

In [42]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [43]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        act_values = self.model.predict(np.array([s],ndmin=2))
        return np.argmax(act_values[0])
        

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        
        for i in range(self.batch_size):
            minibatch=self.memory.random_access()
            s_ = minibatch[0]
            n_s_ = minibatch[1]
            a_ = minibatch[2]   
            r_ = minibatch[3]
            g_over_ = minibatch[4]
            input_states[i] = s_
            s_reshape=np.reshape(s_,(1,5,5,self.n_state))
            n_s_reshape=np.reshape(n_s_,(1,5,5,self.n_state))
            target_q[i] = self.model.predict(s_reshape)
           
            if game_over_:

                target_q[i,a_]=r_
            else:
                Q_sa = self.model.predict(n_s_reshape)
                target_q[i,a_] = r_ + self.discount * np.max(Q_sa)
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
                
        # NN Model
        
        hidden_size= 24
        epsilon = .1  # exploration
        num_actions = 4  # [move_left, move_right, move up, move down]
        max_memory = 5000
        batch_size = 50
        grid_size = 10
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(64, input_shape=(grid_size**2,), activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(num_actions, activation='linear'))
        model.compile(sgd(lr=lr, decay=0.0001, momentum=0.0), "mse")
        self.model=model

In [44]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.2, memory_size=16000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))


Epoch 000/021 | Loss 0.0266 | Win/lose count 3.0/8.0 (-5.0)
Epoch 001/021 | Loss 0.0742 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/021 | Loss 0.1362 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/021 | Loss 0.2501 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/021 | Loss 1.0152 | Win/lose count 1.5/0 (1.5)
Epoch 005/021 | Loss 1.7559 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/021 | Loss 1.8591 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/021 | Loss 1.9956 | Win/lose count 2.5/4.0 (-1.5)
Epoch 008/021 | Loss 2.0126 | Win/lose count 3.0/3.0 (0.0)
Epoch 009/021 | Loss 1.8566 | Win/lose count 5.0/7.0 (-2.0)
Epoch 010/021 | Loss 1.9199 | Win/lose count 0.5/3.0 (-2.5)
Epoch 011/021 | Loss 2.0183 | Win/lose count 2.5/0 (2.5)
Epoch 012/021 | Loss 1.9796 | Win/lose count 3.5/3.0 (0.5)
Epoch 013/021 | Loss 1.8836 | Win/lose count 9.5/6.0 (3.5)
Epoch 014/021 | Loss 1.8509 | Win/lose count 7.0/5.0 (2.0)
Epoch 015/021 | Loss 1.7541 | Win/lose count 4.0/3.0 (1.0)
Epoch 016/021 | Loss 1.7585 | Win/lose count 4.0/3.0

***
***
We implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [45]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super().__init__(*args,**kwargs)
        #super(DQN_CNN, self).__init__(*args,**kwargs)
        
        
        hidden_size= 24
        epsilon = .1  # exploration
        num_actions = 4  # [move_left, move_up, move_down, move_right,]
        max_memory = 500
        batch_size = 50
        grid_size = 10
        
        
        model = Sequential()
        model.add(Conv2D(128, kernel_size=(2, 2),activation='linear',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
        model.add(Dense(24, activation='relu'))
        #model.add(MaxPooling2D(pool_size=(2, 2))
        model.add(Flatten())
        #model.add(Dense(256, activation='relu'))
        #model.add(Dropout(0.5))

        model.add(Dense(num_actions, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [46]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Epoch 000/021 | Loss 0.0975 | Win/lose count 2.0/6.0 (-4.0)
Epoch 001/021 | Loss 1.9298 | Win/lose count 2.5/0 (2.5)
Epoch 002/021 | Loss 2.0264 | Win/lose count 5.5/9.0 (-3.5)
Epoch 003/021 | Loss 0.9790 | Win/lose count 5.5/5.0 (0.5)
Epoch 004/021 | Loss 1.8499 | Win/lose count 6.0/7.0 (-1.0)
Epoch 005/021 | Loss 1.9364 | Win/lose count 6.5/1.0 (5.5)
Epoch 006/021 | Loss 1.9410 | Win/lose count 8.5/3.0 (5.5)
Epoch 007/021 | Loss 1.9171 | Win/lose count 5.5/4.0 (1.5)
Epoch 008/021 | Loss 1.9878 | Win/lose count 6.5/3.0 (3.5)
Epoch 009/021 | Loss 2.0391 | Win/lose count 7.5/5.0 (2.5)
Epoch 010/021 | Loss 1.9924 | Win/lose count 12.5/2.0 (10.5)
Epoch 011/021 | Loss 1.9321 | Win/lose count 6.0/1.0 (5.0)
Epoch 012/021 | Loss 2.0299 | Win/lose count 12.5/1.0 (11.5)
Epoch 013/021 | Loss 1.9129 | Win/lose count 6.5/0 (6.5)
Epoch 014/021 | Loss 1.9133 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/021 | Loss 1.9173 | Win/lose count 10.5/1.0 (9.5)
Epoch 016/021 | Loss 1.9403 | Win/lose count 14.5/1.

In [47]:
HTML(display_videos('cnn_train20.mp4'))

***
***
We test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [ ]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)#change back the temperature to 0.3
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

In [ ]:
HTML(display_videos('cnn_test19.mp4'))

In [ ]:
HTML(display_videos('fc_test19.mp4'))

According to the test results, we observe that the CNN models'average scores are higher than the FC's. We conclude that the CNN model is the best one.
When we increase the temperature, we can observe better results for both models, since we are increasing the percentage of cheeses on the board (binomial distribution).
However, in both models, the agent seems to stay stuck.
He doesn't move a lot from its position, does not explore the board enough. He can't learn well enough.
exploration: The environment is explored, in order to make estimates of rewards for action taken more accurate;
exploitation: Maximization of expected reward, although it can not be done with certainty, because we only have estimates of rewards for actions already taken.
The dilemma is to find the balance between exploring actions that have not been taken before or have been tried very few times to further knowledge and exploiting knowledge already acquired from actions that have been already taken multiple times.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration with the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
We design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    epsilon_decay = 0.995
    
    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy exploration
            agent.set_epsilon(np.float(agent.get_epsilon()/e))

            
            # The agent performs an action
            action = agent.act(state , train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            
        # Save as a mp4
        #if e % 5 == 0:
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4 #increasing plateau of 2 cells
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.malus_position=np.zeros((grid_size,grid_size))
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        #for screening
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends.
        l'environnement recoit l'action et se modifie"""

        self.get_frame(int(self.t))
        
        #reinitialisation de la position du rat à chaque partie mais pas le bord
        self.position = np.zeros((self.grid_size, self.grid_size))
        
        #eau autour du board d'action
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        #differentes positions possibles
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.t = self.t + 1
        #reward =+ self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        #take only the environment of the rat (everything the rat can see)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        #generation de la position du rat à au moins 3 cases de l'eau
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        #bonus for creation of board
        #distribution malus and bonus==> cheese and poison with binomial distribution with temperature 0.5
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        #cas où on aurait -0.5 , cheese preponderant
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
#incorporing an epsilon greedy this algorithm is to decrease the probability as the number of rounds (t) goes up. 
#It’s called epsilon-reducing-greedy algorithm. Intuitively, it makes lot of sense. As the number of rounds go up, the probability of choosing an action randomly goes down i.e., there is less exploration and more exploitation. 
#One way to reduce the probability is as below

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=16000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

***
***
Now we can use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***